In [63]:
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings('ignore')

data_path = 'D:/angel/4th-ML100Days/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = np.log1p(df['Survived'])
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [64]:
df['Ticket'] = df['Ticket'].fillna('None')
df['Age'] = df['Age'].fillna(df['Age'].mean())

mean_df = df.groupby(['Ticket'])['Age'].mean().reset_index()
mode_df = df.groupby(['Ticket'])['Age'].apply(lambda x: x.mode()[0]).reset_index()
median_df = df.groupby(['Ticket'])['Age'].median().reset_index()
max_df = df.groupby(['Ticket'])['Age'].max().reset_index()
min_df = df.groupby(['Ticket'])['Age'].min().reset_index()
temp = pd.merge(mean_df, mode_df, how='left', on=['Ticket'])
temp = pd.merge(temp, median_df, how='left', on=['Ticket'])
temp = pd.merge(temp, max_df, how='left', on=['Ticket'])
temp = pd.merge(temp, min_df, how='left', on=['Ticket'])
temp.columns = ['Ticket', 'Mean', 'Mode', 'Median', 'Max','Min']
temp

,Ticket,Mean,Mode,Median,Max,Min
0,110152,26.333333,16.000000,30.000000,33.000000,16.000000
1,110413,36.333333,18.000000,39.000000,52.000000,18.000000
2,110465,38.349559,29.699118,38.349559,47.000000,29.699118
3,110564,28.000000,28.000000,28.000000,28.000000,28.000000
4,110813,60.000000,60.000000,60.000000,60.000000,60.000000
...,...,...,...,...,...,...
676,W./C. 6608,23.500000,9.000000,18.500000,48.000000,9.000000
677,W./C. 6609,29.699118,29.699118,29.699118,29.699118,29.699118
678,W.E.P. 5734,46.000000,46.000000,46.000000,46.000000,46.000000
679,W/C 14208,30.000000,30.000000,30.000000,30.000000,30.000000


In [65]:
df = pd.merge(df, temp, how='left', on=['Ticket'])
df = df.drop(['Ticket'] , axis=1)
df.head()

num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

10 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Mean', 'Mode', 'Median', 'Max', 'Min']



,Pclass,Age,SibSp,Parch,Fare,Mean,Mode,Median,Max,Min
0,3,22.0,1,0,7.2500,22.0,22.0,22.0,22.0,22.0
1,1,38.0,1,0,71.2833,38.0,38.0,38.0,38.0,38.0
2,3,26.0,0,0,7.9250,26.0,26.0,26.0,26.0,26.0
3,1,35.0,1,0,53.1000,36.0,35.0,36.0,37.0,35.0
4,3,35.0,0,0,8.0500,35.0,35.0,35.0,35.0,35.0


In [67]:
df_minus = df.drop(['Mean', 'Mode', 'Median', 'Max','Min'] , axis=1)

train_X = MMEncoder.fit_transform(df_minus)
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

ValueError: Unknown label type: 'continuous'

In [68]:
train_X = MMEncoder.fit_transform(df)
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

ValueError: Unknown label type: 'continuous'